# Data Preprocessing

In [ ]:
import os
import cv2
import json
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    return img_array

def load_annotations(annotation_path):
    with open(annotation_path, 'r') as f:
        annotations = json.load(f)
    return annotations

def normalize_landmarks(landmarks, normalization_params):
    normalized_landmarks = []

    for landmark_coords in landmarks:
        normalized_coords = []
        for i in range(0, len(landmark_coords), 2):
            x = landmark_coords[i]
            y = landmark_coords[i + 1]

            normalized_x = (x - normalization_params['min_x']) / (normalization_params['max_x'] - normalization_params['min_x'])
            normalized_y = (y - normalization_params['min_y']) / (normalization_params['max_y'] - normalization_params['min_y'])

            normalized_coords.extend([normalized_x, normalized_y])

        normalized_landmarks.append(normalized_coords)

    return np.array(normalized_landmarks)

def prepare_training_data(images_folder, annotations_path, output_folder):
    annotations = load_annotations(annotations_path)

    images = []
    landmarks = []
    filenames = []

    for entry in annotations:
        filename = entry["filename"]
        image_path = os.path.join(images_folder, filename)
        img_array = preprocess_image(image_path)

        # Extract landmark coordinates
        landmark_coords = []
        for landmark_name, coords in entry["landmarks"].items():
            landmark_coords.extend([coords["x"], coords["y"]])

        images.append(img_array)
        landmarks.append(landmark_coords)
        filenames.append(filename)

        # Save the processed image to a new file in the output folder
        output_path = os.path.join(output_folder, f"processed_{filename}")
        cv2.imwrite(output_path, cv2.cvtColor((img_array * 255).astype(np.uint8), cv2.COLOR_RGB2BGR))

    # Calculate normalization parameters
    all_landmarks = np.array(landmarks)
    normalization_params = {
        'min_x': np.min(all_landmarks[:, 0::2]),
        'max_x': np.max(all_landmarks[:, 0::2]),
        'min_y': np.min(all_landmarks[:, 1::2]),
        'max_y': np.max(all_landmarks[:, 1::2]),
    }

    return np.array(images), normalize_landmarks(np.array(landmarks), normalization_params), np.array(filenames)

# Example usage
images_folder = r"/kaggle/input/without-bg-fashion-landmark-detection/FYP Images(BG)/FYP Images(BG)"
annotations_path = r"/kaggle/input/without-bg-fashion-landmark-detection/extracted_annotations.json"
output_folder = r"combined output images"

# Prepare training data
X, y, filenames = prepare_training_data(images_folder, annotations_path, output_folder)

# Split the dataset
X_train, X_temp, y_train, y_temp, filenames_train, filenames_temp = train_test_split(
    X, y, filenames, test_size=0.2, random_state=42
)

X_val, X_test, y_val, y_test, filenames_val, filenames_test = train_test_split(
    X_temp, y_temp, filenames_temp, test_size=0.5, random_state=42
)


# Vgg-16 Model

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import RMSprop

# Load pre-trained VGG-16 model without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Custom layers for landmark detection
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(128))
model.add(LeakyReLU(alpha=0.1))  # Leaky ReLU activation
model.add(layers.Dense(68, activation='linear'))  # Assuming 34 landmarks (x, y coordinates)

# Compile the model
model.compile(optimizer= RMSprop(), loss='mean_squared_error', metrics=['mae'])

# ModelCheckpoint to save the best model during training
checkpoint = ModelCheckpoint("vgg16_landmark_detection_model.h5", save_best_only=True)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=25,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint]
)

# Testing Data

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load your model
model = load_model("vgg16_landmark_detection_model.h5")

# Make predictions on the test dataset
y_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)


# Denormalisation of image and graphical visualization

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
from tensorflow.keras.models import load_model
import random

# Function to load annotations from a JSON file
def load_annotations(annotation_path):
    with open(annotation_path, 'r') as f:
        annotations = json.load(f)
    return annotations

# Function to denormalize landmarks using normalization parameters
def denormalize_landmarks(landmarks, normalization_params):
    denormalized_landmarks = []

    for landmark_coords in landmarks:
        denormalized_coords = []
        for i in range(0, len(landmark_coords), 2):
            normalized_x = landmark_coords[i]
            normalized_y = landmark_coords[i + 1]

            x = normalized_x * (normalization_params['max_x'] - normalization_params['min_x']) + normalization_params['min_x']
            y = normalized_y * (normalization_params['max_y'] - normalization_params['min_y']) + normalization_params['min_y']

            denormalized_coords.extend([x, y])

        denormalized_landmarks.append(denormalized_coords)

    return np.array(denormalized_landmarks, dtype=np.float32)

# Define the path to your annotations file
annotations_path = r'/kaggle/input/landmark-detection-dataset/Combined Annot File.json'

# Load annotations from the JSON file
annotations = load_annotations(annotations_path)

landmarks = []
filenames = []

# Extract landmarks and filenames from annotations
for entry in annotations:
    filename = entry["filename"]

    # Extract landmark coordinates
    landmark_coords = []
    for landmark_name, coords in entry["landmarks"].items():
        landmark_coords.extend([coords["x"], coords["y"]])

    landmarks.append(landmark_coords)
    filenames.append(filename)

# Calculate normalization parameters
all_landmarks = np.array(landmarks)
normalization_params = {
    'min_x': np.min(all_landmarks[:, 0::2]),
    'max_x': np.max(all_landmarks[:, 0::2]),
    'min_y': np.min(all_landmarks[:, 1::2]),
    'max_y': np.max(all_landmarks[:, 1::2]),
}

# Load the model
model = load_model("vgg16_landmark_detection_model.h5")

# Assuming X_test contains your test images and y_test contains your normalized landmarks
# Predict landmarks on the test set
predicted_landmarks_normalized = model.predict(X_test)

# De-normalize both actual and predicted landmarks
denormalized_actual_landmarks = denormalize_landmarks(y_test, normalization_params)
denormalized_predicted_landmarks = denormalize_landmarks(predicted_landmarks_normalized, normalization_params)

# Choose five image indices for visualization
image_indices = random.sample(range(len(filenames_test)), 5)

# Visualization of images with actual and predicted landmarks
for image_index in image_indices:
    # Extract actual and predicted landmarks for the selected image
    actual_landmarks = denormalized_actual_landmarks[image_index]
    predicted_landmarks = denormalized_predicted_landmarks[image_index]

    # Load the image
    image_path = os.path.join(images_folder, filenames_test[image_index])
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Plot the image
    plt.figure()
    plt.imshow(img_rgb)

    # Plot actual landmarks
    plt.scatter(actual_landmarks[0::2], actual_landmarks[1::2], c='r', marker='o', label='Actual Landmarks')

    # Plot predicted landmarks
    plt.scatter(predicted_landmarks[0::2], predicted_landmarks[1::2], c='b', marker='x', label='Predicted Landmarks')
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(actual_landmarks[0::2], label='Actual')
    plt.plot(predicted_landmarks[0::2], label='Predicted', linestyle='dashed')
    plt.title(f"X-coordinate - Filename: {filenames_test[image_index]}")
    plt.legend()

    # Plot the graph for y-coordinates
    plt.subplot(1, 2, 2)
    plt.plot(actual_landmarks[1::2], label='Actual')
    plt.plot(predicted_landmarks[1::2], label='Predicted', linestyle='dashed')
    plt.title(f"Y-coordinate - Filename: {filenames_test[image_index]}")
    plt.legend()
    
    plt.title(f"Image: {filenames_test[image_index]}")
    plt.legend()
    plt.show()